<h1 style="text-align:center">Deep Learning   </h1>
<h1 style="text-align:center"> Lab Session 2 - 3 Hours </h1>
<h1 style="text-align:center"> Convolutional Neural Network (CNN) for Handwritten Digits Recognition</h1>

<b> Student 1:</b> Matteo Fiore 
<b> Student 2:</b> Hanna Johansson
 
 
The aim of this session is to practice with Convolutional Neural Networks. Answers and experiments should be made by groups of one or two students. Each group should fill and run appropriate notebook cells. 


Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an pdf document using print as PDF (Ctrl+P). Do not forget to run all your cells before generating your final report and do not forget to include the names of all participants in the group. The lab session should be completed by May 29th 2017.

Send you pdf file to benoit.huet@eurecom.fr and olfa.ben-ahmed@eurecom.fr using **[DeepLearning_lab2]** as Subject of your email.

# Introduction

In the last Lab Session, you built a Multilayer Perceptron for recognizing hand-written digits from the MNIST data-set. The best achieved accuracy on testing data was about 97%.  Can  you do better than these results using a deep CNN ?
In this Lab Session, you will build, train and optimize in TensorFlow one of the early Convolutional Neural Networks:  **LeNet-5** to go to  more than 99% of accuracy. 






# Load MNIST Data in TensorFlow
Run the cell above to load the MNIST data that comes  with TensorFlow. You will use this data in **Section 1** and **Section 2**.

In [113]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
X_train, y_train           = mnist.train.images, mnist.train.labels
X_validation, y_validation = mnist.validation.images, mnist.validation.labels
X_test, y_test             = mnist.test.images, mnist.test.labels
print("Image Shape: {}".format(X_train[0].shape))
print("Training Set:   {} samples".format(len(X_train)))
print("Validation Set: {} samples".format(len(X_validation)))
print("Test Set:       {} samples".format(len(X_test)))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Image Shape: (784,)
Training Set:   55000 samples
Validation Set: 5000 samples
Test Set:       10000 samples


# Section 1 : My First Model in TensorFlow

Before starting with CNN, let's train and test in TensorFlow the example :
**y=softmax(Wx+b)** seen in the DeepLearing course last week. 

This model reaches an accuracy of about 92 %.
You will also learn how to launch the tensorBoard https://www.tensorflow.org/get_started/summaries_and_tensorboard to  visualize the computation graph, statistics and learning curves. 

<b> Part 1 </b> : Read carefully the code in the cell below. Run it to perform training. 

In [4]:
from __future__ import print_function
import tensorflow as tf


#STEP 1

# Parameters
learning_rate = 0.01
training_epochs = 100
batch_size = 128
display_step = 1
logs_path = 'log_files/'  # useful for tensorboard

tf.reset_default_graph()
# tf Graph Input:  mnist data image of shape 28*28=784
x = tf.placeholder(tf.float32, [None, 784], name='InputData')
# 0-9 digits recognition,  10 classes
y = tf.placeholder(tf.float32, [None, 10], name='LabelData')

# Set model weights
W = tf.Variable(tf.zeros([784, 10]), name='Weights')
b = tf.Variable(tf.zeros([10]), name='Bias')

# Construct model and encapsulating all ops into scopes, making Tensorboard's Graph visualization more convenient
with tf.name_scope('Model'):
    # Model
    pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax
with tf.name_scope('Loss'):
    # Minimize error using cross entropy
    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
with tf.name_scope('SGD'):
    # Gradient Descent
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
with tf.name_scope('Accuracy'):
    # Accuracy
    acc = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()
# Create a summary to monitor cost tensor
tf.summary.scalar("Loss", cost)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("Accuracy", acc)
# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()


#STEP 2 


# Launch the graph for training
with tf.Session() as sess:
    sess.run(init)
    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop), cost op (to get loss value)
            # and summary nodes
            _, c, summary = sess.run([optimizer, cost, merged_summary_op],
                                     feed_dict={x: batch_xs, y: batch_ys})
            # Write logs at every iteration
            summary_writer.add_summary(summary, epoch * total_batch + i)
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print("Epoch: ", '%02d' % (epoch+1), "  =====> Loss=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")

    # Test model
    # Calculate accuracy
    print("Accuracy:", acc.eval({x: mnist.test.images, y: mnist.test.labels}))




Epoch:  01   =====> Loss= 1.287774475
Epoch:  02   =====> Loss= 0.733350185
Epoch:  03   =====> Loss= 0.599991729
Epoch:  04   =====> Loss= 0.535803187
Epoch:  05   =====> Loss= 0.498761953
Epoch:  06   =====> Loss= 0.472009526
Epoch:  07   =====> Loss= 0.450981238
Epoch:  08   =====> Loss= 0.433896199
Epoch:  09   =====> Loss= 0.422335408
Epoch:  10   =====> Loss= 0.413419973
Epoch:  11   =====> Loss= 0.407670022
Epoch:  12   =====> Loss= 0.396227188
Epoch:  13   =====> Loss= 0.389944836
Epoch:  14   =====> Loss= 0.383488354
Epoch:  15   =====> Loss= 0.382057854
Epoch:  16   =====> Loss= 0.371171829
Epoch:  17   =====> Loss= 0.370542881
Epoch:  18   =====> Loss= 0.366848597
Epoch:  19   =====> Loss= 0.363278455
Epoch:  20   =====> Loss= 0.361297389
Epoch:  21   =====> Loss= 0.354854941
Epoch:  22   =====> Loss= 0.353361876
Epoch:  23   =====> Loss= 0.350632020
Epoch:  24   =====> Loss= 0.348237960
Epoch:  25   =====> Loss= 0.347620594
Epoch:  26   =====> Loss= 0.346936341
Epoch:  27  

<b> Part 2  </b>: Using Tensorboard, we can  now visualize the created graph, giving you an overview of your architecture and how all of the major components  are connected. You can also see and analyse the learning curves. 

To launch tensorBoard: 
- Go to the **TP2** folder, 
- Open a Terminal and run the command line **"tensorboard --logdir= log_files/"**, it will generate an http link --> http://192.168.17.2:6006/ 
- Copy this  link into your web browser 


Enjoy It !! 


# Section 2 : The 99% MNIST Challenge !

<b> Part 1 </b> : LeNet5 implementation

Once you are now familar with **tensorFlow** and **tensorBoard**, you are in this section to build, train and test the baseline [LeNet-5](http://yann.lecun.com/exdb/lenet/)  model for the MNIST digits recognition problem.  

In more advanced step you will make some optimizations to get more than 99% of accuracy. The best model can get to over 99.7% accuracy! 

For more information, have a look at this list of results : http://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results.html


<img src="lenet.png",width="800" height="600" align="center">
<center><span>Figure 1: Lenet 5 </span></center>





The LeNet architecture accepts a 32x32xC image as input, where C is the number of color channels. Since MNIST images are grayscale, C is 1 in this case.

--------------------------
**Layer 1: Convolutional.** The output shape should be 28x28x6 **Activation.** sigmoid **Pooling.** The output shape should be 14x14x6.

**Layer 2: Convolutional.** The output shape should be 10x10x16. **Activation.** sigmoid **Pooling.** The output shape should be 5x5x16.

**Flatten.** Flatten the output shape of the final pooling layer such that it's 1D instead of 3D.  You may need to use **flatten*  from tensorflow.contrib.layers import flatten

**Layer 3: Fully Connected.** This should have 120 outputs. **Activation.** sigmoid

**Layer 4: Fully Connected.** This should have 84 outputs. **Activation.** sigmoid

**Layer 5: Fully Connected.** This should have 10 outputs. **Activation.** softmax


<b> Question 2.1.1 </b>  Implement the Neural Network architecture described above.
For that, your will use classes and functions from  https://www.tensorflow.org/api_docs/python/tf/nn. 

We give you some helper functions for weigths and bias initilization. Also you can refer to section 1. 


In [80]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

In [114]:
def LeNet5_Model(data):    
    ######## Convolutional Layer #1
    
    # [filter_height,filter_width, in_channel, num_out_filters]
    filter_shape_c1 = [5, 5, 1, 6]
    f_conv1 = weight_variable(filter_shape_c1)
    
    conv1 = tf.nn.conv2d(input=data, filter=f_conv1, strides=[1,1,1,1], padding="SAME")
    print(conv1.shape,'conv1')
    # (?, 28, 28, 6) conv1
    
    # activation 
    b1 = bias_variable([6])
    h1 = tf.nn.sigmoid(tf.nn.bias_add(conv1,b1))
    print(h1.shape,'h1')
    # (?, 28, 28, 6) h1
    
    # Pooling layer 1
    pooled1 = tf.nn.max_pool(h1,ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
    print(pooled1.shape,'pooled1')
    # (?, 14, 14, 6) pooled1
    
    ######### Convolutional Layer #2
    
    # [filter_height,filter_width, in_channel, num_out_filters]
    filter_shape_c2 = [5,5,6,16]
    f_conv2 = weight_variable(filter_shape_c2)
    
    conv2 = tf.nn.conv2d(pooled1, filter=f_conv2, strides=[1,1,1,1], padding="VALID")
    print(conv2.shape,'conv2')
    # (?, 14, 14, 16) conv2
    
    # activation 
    b2 = bias_variable([16])
    h2 = tf.nn.sigmoid(tf.nn.bias_add(conv2,b2))
    print(h2.shape,'h2')
    # (?, 14, 14, 16) h2
    
    # Pooling layer 1
    pooled2 = tf.nn.max_pool(h2,ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
    print(pooled2.shape,'pooled2')
    # (?, 7, 7, 16) pooled2
    
    pooled2_flat = tf.reshape(pooled2, [-1, 5*5*16])
    
    dense1 = tf.layers.dense(inputs=pooled2_flat, units=120, activation=tf.nn.sigmoid)
    dense2 = tf.layers.dense(inputs=dense1, units=84, activation=tf.nn.sigmoid)
    dense3 = tf.layers.dense(inputs=dense2, units=10, activation=tf.nn.softmax)
    
    return dense3








<b> Question 2.1.2. </b>  Calculate the number of parameters of this model 

 Your answer goes here in details 

<b> Question 2.1.3. </b>  Start the training with the parameters cited below:

     Learning rate =0.1
     Loss Fucntion : Cross entropy
     Optimisateur: SGD
     Number of training iterations= 10000
     The batch size =128

In [115]:
# Training parameters 
learning_rate = 0.1
training_epochs = 10000
batch_size = 128
optimizer = tf.train.GradientDescentOptimizer

<b> Question 2.1.4. </b>  Implement the evaluation function for accuracy computation 

In [116]:
def evaluate(model, y):
    acc = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))
    return accuracy

In [117]:
y_test.shape

(10000, 10)

<b> Question 2.1.5. </b>  Implement training pipeline and run the training data through it to train the model.

- Before each epoch, shuffle the training set. 
- Print the loss per mini batch and the training/validation accuracy per epoch. (Display results every 100 epochs)
- Save the model after training
- Print after training the final testing accuracy 



In [120]:

# Initializing the variables
def train(learning_rate = 0.1, training_epochs = 50, batch_size = 128, display_step = 1, 
            optimizer= tf.train.GradientDescentOptimizer):
    # Initializing the session 
    
    tf.reset_default_graph()
    print ("Start Training!")     
        
    # tf Graph Input:  mnist data image of shape 28*28=784
    x = tf.placeholder(tf.float32, [None, 28, 28, 1], name='InputData')
    # 0-9 digits recognition,  10 classes
    y = tf.placeholder(tf.float32, [None, 10], name='LabelData')

    # Construct model and encapsulating all ops into scopes, making Tensorboard's Graph visualization more convenient
    with tf.name_scope('Model'):
        # Model
        pred = LeNet5_Model(x) # Softmax
    with tf.name_scope('Loss_LaNet'):
        # Minimize error using cross entropy
        cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
    with tf.name_scope('SGD'):
        # Gradient Descent
        optimizer = optimizer(learning_rate).minimize(cost)
    with tf.name_scope('Accuracy_LaNet'):
        # Accuracy
        acc = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
        acc = tf.reduce_mean(tf.cast(acc, tf.float32))

    # Initializing the variables
    init = tf.global_variables_initializer()
    # Create a summary to monitor cost tensor
    tf.summary.scalar("Loss_LaNet", cost)
    # Create a summary to monitor accuracy tensor
    tf.summary.scalar("Accuracy_LaNet", acc)
    # Merge all summaries into a single op
    merged_summary_op = tf.summary.merge_all()


    # Launch the graph for training
    with tf.Session() as sess:
        sess.run(init)
        # op to write logs to Tensorboard
        summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
        # Training cycle
        for epoch in range(training_epochs):
            avg_cost = 0.
            total_batch = int(mnist.train.num_examples/batch_size)
            # Loop over all batches
            for i in range(total_batch):
                batch_xs, batch_ys = mnist.train.next_batch(batch_size)
                shuffle= np.arange(batch_size)
                np.random.shuffle(shuffle)
                batch_xs = batch_xs[shuffle]
                batch_ys = batch_ys[shuffle]
                batch_xs = batch_xs.reshape([-1, 28, 28, 1])
                # Run optimization op (backprop), cost op (to get loss value)
                # and summary nodes
                _, c, summary = sess.run([optimizer, cost, merged_summary_op],
                                         feed_dict={x: batch_xs, y: batch_ys})
                # Write logs at every iteration
                summary_writer.add_summary(summary, epoch * total_batch + i)
                # Compute average loss
                avg_cost += c / total_batch
            # Display logs per epoch step
            if (epoch+1) % display_step == 0:
                print("Epoch: ", '%02d' % (epoch+1), "  =====> Loss=", "{:.9f}".format(avg_cost))
                print("\t\t======> Training Accuracy:", acc.eval({x: mnist.train.images.reshape([-1,28,28,1]), y: mnist.train.labels}))
                print("\t\t======> Validation Accuracy:", acc.eval({x: mnist.validation.images.reshape([-1,28,28,1]), y: mnist.validation.labels}))
        
        
        print ("Training Finished!")
        
        #Your implementation for testing accuracy after training goes here
        print("Testing Accuracy:", acc.eval({x: mnist.test.images.reshape([-1,28,28,1]), y: mnist.test.labels}))
        
        
    

In [122]:
train(learning_rate = learning_rate, training_epochs = 2, batch_size = 128, display_step = 1, 
            optimizer= tf.train.GradientDescentOptimizer)

Start Training!
(?, 28, 28, 6) conv1
(?, 28, 28, 6) h1
(?, 14, 14, 6) pooled1
(?, 10, 10, 16) conv2
(?, 10, 10, 16) h2
(?, 5, 5, 16) pooled2
Epoch:  01   =====> Loss= 2.308882457
		======> Training Accuracy: 0.112345
		======> Validation Accuracy: 0.1126
Epoch:  02   =====> Loss= 2.305897100
		======> Training Accuracy: 0.112345
		======> Validation Accuracy: 0.1126
Training Finished!
Testing Accuracy: 0.1135


<b> Question 2.1.6 </b> : Use tensorBoard to visualise and save the LeNet5 Graph and all learning curves. 
Save all obtained figures in the folder **"TP2/MNIST_99_Challenge_Figures"**

In [ ]:
#  insert your obtained figure here 

In [ ]:
# your answer goas here

<b> Part 2 </b> : LeNET 5 Optimization


<b> Question 2.2.1 </b>  Change the sigmoid function with a Relu :

- Retrain your network with SGD and AdamOptimizer and then fill the table above  :


| Optimizer            |  Gradient Descent         |AdamOptimizer |
| -------------        |: -------------: | ---------:   
| Validation Accuracy  |         |    |      
| Testing Accuracy     |           |    |       
| Training Time        |           |        |  |  


- Try with different learning rates for each Optimizer (0.0001 and 0.001 ) and different Batch sizes (50 and 128) for 20000 Epochs. 

- For each optimizer, plot (on the same curve) the **testing accuracies** function to **(learning rate, batch size)** 



- Did you reach the 99% accuracy ? What are the optimal parametres that gave you the best results? 








In [ ]:
# your answer goas here

<b> Question 2.2.2 </b>  What about applying a dropout layer on the Fully conntected layer and then retraining the model with the best Optimizer and parameters(Learning rate and Batsh size) obtained in *Question 2.2.1*  ? (probability to keep units=0.75). For this stage ensure that the keep prob is set to 1.0 to evaluate the 
performance of the network including all nodes.

In [ ]:
# your implementaion goas here

In [ ]:
Your comments go here